In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import pandas as pd
import numpy as np

# Import csv from preprocessing - generated with python script

In [2]:
test = pd.read_csv('../identification.csv')

## Preprocess data from csv/DataFrame

In [3]:
sent_x = []
sent_y = []
for p in set(test.paragraph.values):
    for s in set(test[test.paragraph==p].sentence.values):
        temp_test = test[(test.paragraph == p) & (test.sentence == s)]
        sent_x.append(temp_test.loc[:, temp_test.columns != 'IOB'].to_dict("record"))
        sent_y.append(temp_test.IOB.values)

/var/folders/7r/c0g604c13xn1wrqn69g540c00000gn/T/ipykernel_64815/1349379417.py:6: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  sent_x.append(temp_test.loc[:, temp_test.columns != 'IOB'].to_dict("record"))


## Split data - informal for now and only on esssay001

In [4]:
X_train = sent_x[:10]
y_train = sent_y[:10]
X_test = sent_x[10:]
y_test = sent_y[10:]

## Initialize CRF trainer

In [5]:
trainer = pycrfsuite.Trainer(verbose=False)


### add training data

In [6]:
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [7]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

## Train and save to file

In [8]:
trainer.train('conll2002-esp.crfsuite')

## Load model from file

In [9]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

## Test

In [10]:
print("Predicted:", ' '.join(tagger.tag(X_test[2])))
print("Correct:  ", ' '.join(y_test[2]))

Predicted: Arg-B Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I O
Correct:   Arg-B Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I O


In [11]:
y_pred = []
for i, each in enumerate(X_test):
    y_pred.append(tagger.tag(each))


In [12]:
lb = LabelBinarizer()

In [13]:
y_true_combined = lb.fit_transform(list(chain.from_iterable(y_test)))

In [14]:
y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))


In [15]:
   
tagset = set(lb.classes_) - {'O'}
tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

report = classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
)

/Users/kevinyin/opt/anaconda3/envs/experiments/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kevinyin/opt/anaconda3/envs/experiments/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
print(report)

              precision    recall  f1-score   support

       Arg-B       0.50      0.50      0.50         4
       Arg-I       0.87      0.88      0.88       102

   micro avg       0.86      0.87      0.86       106
   macro avg       0.69      0.69      0.69       106
weighted avg       0.86      0.87      0.86       106
 samples avg       0.56      0.56      0.56       106

